In [1]:
"""
Extract gridMET variables for daily burn perimeters, western U.S.
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import ee
import geemap
import time

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')
print("GEE Authenticated !")

GEE Authenticated !


In [ ]:
# Function to calculate gridmet for daily perimeters
def gridmet_summary(fires, reducer, attrs, buffer=None):
    """ Calculates the gridMET variables for a given daily perimeter by daily ID 
    Args:
        - fires: fire perimeters (with daily polygons)
        - reducer: the reduction to use for the gridmet variables
        - attrs: the gridmet variables
        - buffer: (optional) apply a buffer to the daily polygon
    """
    # Set a function to calculate the variables for each daily perimeter
    def get_daily_gridmet(perim):
        # Buffer if requested
        if buffer is not None:
            geom = perim.geometry().bounds().buffer(buffer)
        else:
            geom = perim.geometry().bounds()
        did = perim.get('did') # get the daily burn ID
        # Get the burn date for extracting gridMET variables
        burn_date = ee.Date(perim.get('date')) 
        gridmet_ = gridmet.filterDate(burn_date, burn_date.advance(1, 'day')).select(attrs)
        gridmet_mn = gridmet_.mean() # take the mean of the imageCollection (daily gridmets)
        # Set up the feature reduction
        stats = gridmet_mn.reduceRegion(
            reducer=reducer,
            geometry=geom,
            scale=4000,  # Match gridMET resolution
            bestEffort=True)
        return perim.set(stats)
    results = fires.map(get_daily_gridmet)
    return results

def parse_date(ftr, date_col):
    """ Function to safely parse a date column and raise error if the date is invalid """
    try:
        date_str = ee.String(ftr.get(date_col)).split(' ').get(0) # removes the time stamp
        # Try to parse the date using the provided format
        date = ee.Date.parse('YYYY/MM/dd', date_str)
        return ftr.set('date', date)
    except ee.EEException as e:
        # Raise an error or skip the feature if the date is invalid
        print(f"Error parsing date for feature with ID {ftr.get('did').getInfo()}: {str(e)}")
        return None  # Skip features with invalid dates

def check_dates(fc, date_col):
    """ Filters out features with invalid dates """
    valid_dates = fc.map(lambda ftr: parse_date(ftr, date_col)).filter(ee.Filter.notNull(['date']))
    return valid_dates

def monitor_export(task):
    """ Monitors EE export task """
    while task.active():
        print('Waiting for export to finish...')
        time.sleep(30)  # Check every 30 seconds
    
    # Get the status of the task
    status = task.status()
    
    # Check if the task failed or succeeded
    if status['state'] == 'COMPLETED':
        print("Export completed successfully!")
    elif status['state'] == 'FAILED':
        print(f"Export failed! Reason: {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Export ended with state: {status['state']}")

print("Functions loaded !")

In [14]:
# Load the gridMET datasets
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"gridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}")

gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']


In [ ]:
# Implement the function

In [31]:
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/fired-daily_west_2018_to_2024')
print(f"There are [{fires.size().getInfo()}] daily fire perimeters.")

# Parse the date column:
fires = check_dates(fires, 'date')
print(f"Successfully parsed dates for [{fires.size().getInfo()}] daily perimeters.")
print(f"Example date: {fires.first().get('date').getInfo()}")

There are [32493] daily fire perimeters.
Successfully parsed dates for [32493] daily perimeters.
Example date: {'type': 'Date', 'value': 1582934400000}


In [32]:
# Define the function parameters
reduction = ee.Reducer.percentile([95]) # 95th percentile
attrs = ['fm1000', 'vpd', 'erc', 'bi', 'tmmx', 'rmin', 'vs'] # the gridmet attributes to use
# Retrieve gridmet summaries for daily perimeters
results_fc = gridmet_summary(fires, reduction, attrs)
print("Success !")

Success !


In [33]:
# Check the results
results_fc.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-118.99755710339323, 35.05417427658838],
    [-118.99145840860311, 35.04999004807538],
    [-118.98634903602644, 35.04999023890428],
    [-118.9924474889566, 35.054174741246676],
    [-118.99755710339323, 35.05417427658838]]]},
 'id': '00000000000000004264',
 'properties': {'bi': 45,
  'date': {'type': 'Date', 'value': 1582934400000},
  'did': '0b908840d20743a1554d1ca3e1d42df8',
  'dy_ar_km2': 0.214658673296478,
  'eco_mode': 1,
  'eco_name': 'California Central Valley grasslands',
  'eco_type': 'WWF Terrestrial Ecoregions of the World',
  'erc': 52,
  'event_day': 1,
  'event_dur': 1,
  'fid': 1441,
  'fm1000': 12.899999618530273,
  'fsr_km2_dy': 0.214658673296478,
  'fsr_px_dy': 1,
  'id': 5247,
  'ig_date': '2020/02/29 00:00:00.000',
  'ig_day': '060',
  'ig_month': 2,
  'ig_utm_x': 3897618.156358264,
  'ig_utm_y': -10831556.656358264,
  'ig_year': 2020,
  'last_date': '2020/02/29 00:00:00.000',
  'lc_code': 13

In [39]:
# Subset and export (gridmet attributes and DID)
print(f"Export columns: {['did'] + attrs}")
results_fc_ = results_fc.select(attrs + ['did'])

Export columns: ['did', 'fm1000', 'vpd', 'erc', 'bi', 'tmmx', 'rmin', 'vs']


In [40]:
# Export the table to an asset
export_task = ee.batch.Export.table.toAsset(
    collection=results_fc_,
    description='fired-daily_west_2018_to_2024_gridmet',
    assetId='projects/jfsp-aspen/assets/fired-daily_west_2018_to_2024_gridmet' 
)
# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task)

Export to Earth Engine Asset started!


In [ ]:
# Monitor the task until it's finished
while export_task.active():
    print('Waiting for export to finish...')
    time.sleep(60)
print('Export finished!')

Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...
Waiting for export to finish...


In [ ]:
# Export the FeatureCollection to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=results_fc_,
    description='fired-daily_west_2018_to_2024_gridmet',
    fileFormat='CSV'
)
# Start the export task
export_task.start()
print("Export to Google Drive started!")
# Monitor the task until it's finished
monitor_export(export_task)